## 📊 Técnicas de Validação de Modelos

As **técnicas de validação de modelos** são métodos usados para **avaliar o desempenho de um modelo de machine learning** em dados **não vistos**, ou seja, dados que não foram usados durante o treinamento. O objetivo é **estimar como o modelo se comportará no mundo real**, com dados novos.

### 🧠 Por que validar um modelo?

Quando criamos um modelo de machine learning, queremos ter certeza de que ele:

- **Aprendeu bem os padrões dos dados de treino** (sem memorizar),
- **Generaliza bem para dados novos** (não caiu em overfitting),
- **Tem bom desempenho em situações reais**.

A validação nos ajuda a responder perguntas como:
> "Esse modelo vai performar bem quando eu colocá-lo em produção?"

### 🧪 Exemplo Geral

Vamos usar o famoso conjunto de dados **Iris**, que é perfeito para demonstrações rápidas:

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Carregando os dados
iris = load_iris()
X, y = iris.data, iris.target

Usaremos um modelo simples de classificação: **Random Forest**.

In [2]:
model = RandomForestClassifier(random_state=42)

### 🔍 Principais Técnicas de Validação

Vamos conhecer as técnicas mais comuns, explicadas de forma simples:

#### 1. Holdout (Divisão treino/teste/validação)

- É a técnica mais simples.
- Divide os dados em duas partes:
  - **Treino** (ex: 70% dos dados) – usado para treinar o modelo,
  - **Teste** (ex: 30%) – usado para avaliar o modelo.

✅ **Vantagem**: rápida e fácil  
❌ **Desvantagem**: pode ser imprecisa se a divisão for ruim (ex: dados desbalanceados)

- Exemplo: 

    ```python
    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
    ```

In [3]:
# Divisão treino-teste (70% treino, 30% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Treinando o modelo
model.fit(X_train, y_train)

# Fazendo previsões
y_pred = model.predict(X_test)

# Avaliando
print("Acurácia (Holdout):", accuracy_score(y_test, y_pred))

Acurácia (Holdout): 1.0


### 2. Validação Cruzada (k-fold)

- Divide os dados em **K partes (folds)** (por exemplo, K=5).
- O modelo é treinado e testado **K vezes**, sempre usando uma parte diferente para teste e as demais para treino.
- No final, calcula-se a média das métricas obtidas.
- Mais confiável que o Holdout. Divide os dados em partes iguais.

✅ **Vantagem**: uso mais eficiente dos dados, resultado mais confiável  
❌ **Desvantagem**: mais lento que o Holdout

Exemplo comum: **K-Fold Cross Validation**

- Exemplo:
    ```python 
    from sklearn.model_selection import cross_val_score
    from sklearn.ensemble import RandomForestClassifier

    scores = cross_val_score(RandomForestClassifier(), X, y, cv=5)
    print("Acurácia média:", scores.mean())
    ```

In [4]:
from sklearn.model_selection import cross_val_score

# Validação cruzada com 5 folds
scores = cross_val_score(model, X, y, cv=5)

print("Acurácias em cada fold:", scores)
print("Média da acurácia:", scores.mean())

Acurácias em cada fold: [0.96666667 0.96666667 0.93333333 0.96666667 1.        ]
Média da acurácia: 0.9666666666666668


### 3. Leave-One-Out


- Uma variação extrema da validação cruzada.
- Cada amostra do conjunto de dados é usada **uma vez como dado de teste**, enquanto o restante é usado para treino.
- Muito custoso, mas útil em conjuntos pequenos.
- Muito precisa, mas custoso computacionalmente.

✅ Muito precisa (com grandes conjuntos de dados)  
❌ Muito custosa computacionalmente


- Exemplo:
    ```python
    from sklearn.model_selection import LeaveOneOut

    loo = LeaveOneOut()
    scores = cross_val_score(model, X, y, cv=loo)
    ```

In [6]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
scores = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))

print("Média da acurácia (LOO):", np.mean(scores))

Média da acurácia (LOO): 0.9533333333333334



### 4. Validação Estratificada - Stratified K-Fold (para dados desbalanceados)

- Similar ao K-Fold, mas mantém a **distribuição das classes** em cada fold.
- Útil em problemas de classificação com dados **desbalanceados**.
- Garante distribuição proporcional das classes nos folds.
- Mantém a proporção das classes em cada fold. Ideal para classificação desbalanceada.

- Exemplo:
    ```python
    from sklearn.model_selection import StratifiedKFold

    skf = StratifiedKFold(n_splits=5)
    ```

In [5]:
from sklearn.model_selection import StratifiedKFold

# Dados artificiais desbalanceados para exemplo
import numpy as np
y_desbalanceado = np.array([0]*90 + [1]*10)  # 90 zeros e 10 uns
X_desbalanceado = X[:100]

# Stratified K-Fold
skf = StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(X_desbalanceado, y_desbalanceado):
    X_train, X_test = X_desbalanceado[train_index], X_desbalanceado[test_index]
    y_train, y_test = y_desbalanceado[train_index], y_desbalanceado[test_index]
    model.fit(X_train, y_train)
    print("Acurácia (Stratified K-Fold):", model.score(X_test, y_test))

Acurácia (Stratified K-Fold): 0.9
Acurácia (Stratified K-Fold): 0.9
Acurácia (Stratified K-Fold): 0.95
Acurácia (Stratified K-Fold): 0.7
Acurácia (Stratified K-Fold): 0.85


### 5. Validação Temporal (Séries Temporais)

- Específica para **séries temporais**.
- Garante que os dados de teste sempre sejam **posteriores aos de treino** (evita vazamento temporal).
- Importante para não embaralhar dados dependentes do tempo.

- Exemplo:
    ```python

    from sklearn.model_selection import TimeSeriesSplit
    
    tscv = TimeSeriesSplit(n_splits=5)
    
```

In [8]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Dados simulados como série temporal
X_ts = np.sort(np.random.rand(100, 1), axis=0)
y_ts = np.sin(X_ts).ravel() + np.random.normal(0, 0.1, 100)

# Modelo de regressão
model = RandomForestRegressor(random_state=42)

# Validação para séries temporais
tscv = TimeSeriesSplit(n_splits=5)

for train_index, test_index in tscv.split(X_ts):
    X_train, X_test = X_ts[train_index], X_ts[test_index]
    y_train, y_test = y_ts[train_index], y_ts[test_index]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print("R² Score (TimeSeriesSplit):", r2_score(y_test, y_pred))

R² Score (TimeSeriesSplit): -0.0022504496585205036
R² Score (TimeSeriesSplit): -1.9519367995995807
R² Score (TimeSeriesSplit): -2.1442985758350996
R² Score (TimeSeriesSplit): -3.389018509057993
R² Score (TimeSeriesSplit): -2.6420987245118335


### 📊 Métricas Comuns Usadas na Validação

Dependendo do tipo de problema, usamos diferentes métricas para avaliar o modelo:

| Tipo de Problema | Métricas Comuns |
|------------------|-----------------|
| Classificação    | Acurácia, F1 Score, Precision, Recall, ROC-AUC |
| Regressão        | MAE, MSE, RMSE, R² |
| Clusterização    | Silhouette Score, Rand Index |

### 💡 Dica: Não use apenas acurácia!

- Muitas vezes, a **acurácia** pode enganar (principalmente em dados desbalanceados). 
- Use outras métricas como **F1-score** ou **ROC-AUC** para ter uma visão mais completa do desempenho do seu modelo.

### ✅ Resumo Final

| Técnica                | Quando usar?                             |
|------------------------|------------------------------------------|
| Holdout                | Quando tem muitos dados e pouco tempo    |
| K-Fold Cross Validation| Para estimativa mais robusta do desempenho |
| Stratified K-Fold      | Em classificações com classes desbalanceadas |
| Leave-One-Out          | Com poucos dados e alta necessidade de precisão |
| Time Series Split      | Em séries temporais                      |